In [1]:
# Importing Libraries/Dependencies
import selenium
import pandas as pd
import time
import requests
from selenium import webdriver
from  bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

## Scraping Reviews from https://www.amazon.in/

In [2]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

<ipython-input-2-2246cf2d3da3>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


In [3]:
# Getting the webpage of mentioned url
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)

In [4]:
# Listing down the items that need to be searched
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']

In [5]:
# Creating empty lists to fetch required data
title = []           # Title of the product
review_text = []     # Text of the review or comments of the particular product
ratings = []         # Rating of the product

In [6]:
# Fetching the required data from the website
for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  # Clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 

<ipython-input-6-38f771eabcfc>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  srchbar = driver.find_element_by_id("twotabsearchtextbox")
<ipython-input-6-38f771eabcfc>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("nav-search-submit-button").click()  # Clicking on search button
<ipython-input-6-38f771eabcfc>:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
<ipython-input-6-38f771eabcfc>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
<ipython-input-6-38f771eabcfc>:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_elem

In [7]:
len(ratings), len(title), len(review_text)

(75918, 75918, 75918)

In [8]:
#creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df

,Review_title,Reiew_text,Ratings
0,",how do I activate ms office lifetime0l",",how do I activate ms office lifetime",3.0 out of 5 stars
1,Bluetooth not working,Bluetooth not working,1.0 out of 5 stars
2,Hangs soo much that you will feel frustrated,"Hangs soo much that you feel frustrated,<br>La...",1.0 out of 5 stars
3,Running smoothly,No heating issue was found....!!<br>working ve...,5.0 out of 5 stars
4,A beast in the budget price,I purchased this laptop using exchange offer a...,5.0 out of 5 stars
...,...,...,...
75913,Good,Good,4.0 out of 5 stars
75914,Best product in this price range,Gud,4.0 out of 5 stars
75915,Value for money,Good product in good price,4.0 out of 5 stars
75916,Good product,Good product.,4.0 out of 5 stars


In [9]:
# Saving the data into csv file
df.to_csv("Amazon_reviews.csv")

In [10]:
# lets load the flipkart reviews file
flpkrt = pd.read_csv("Flipkrt_reviews.csv")
flpkrt

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Terrific,Hp laptop is the Best one 👍💞😍\nI am so Happy f...,5
1,1,Mind-blowing purchase,Great laptop\nmust buy\nPerfect for multimedia...,5
2,2,Excellent,Good laptop for students and day today work . ...,5
3,3,Perfect product!,Just awesome laptop... Everything is just perf...,5
4,4,Decent product,Not Worth As per price and after running this ...,3
...,...,...,...,...
32380,32380,Worthless,One of the worst products the screen protector...,1
32381,32381,Wonderful,Product quality is normal.,4
32382,32382,Wonderful,Product quality is normal.,4
32383,32383,Nice product,Awsm\nSticker is good\nCover is good\nKeyboard...,4


In [11]:
# Concatenating the two datasets 
reviews = pd.concat([df, flpkrt],ignore_index=True)
reviews

,Review_title,Reiew_text,Ratings,Unnamed: 0
0,",how do I activate ms office lifetime0l",",how do I activate ms office lifetime",3.0 out of 5 stars,NaN
1,Bluetooth not working,Bluetooth not working,1.0 out of 5 stars,NaN
2,Hangs soo much that you will feel frustrated,"Hangs soo much that you feel frustrated,<br>La...",1.0 out of 5 stars,NaN
3,Running smoothly,No heating issue was found....!!<br>working ve...,5.0 out of 5 stars,NaN
4,A beast in the budget price,I purchased this laptop using exchange offer a...,5.0 out of 5 stars,NaN
...,...,...,...,...
108298,Worthless,One of the worst products the screen protector...,1,32380.0
108299,Wonderful,Product quality is normal.,4,32381.0
108300,Wonderful,Product quality is normal.,4,32382.0
108301,Nice product,Awsm\nSticker is good\nCover is good\nKeyboard...,4,32383.0


In [12]:
# Dropping unwanted column
reviews.drop(columns = 'Unnamed: 0', inplace=True)

In [13]:
# Displaying the merged dataset
reviews

,Review_title,Reiew_text,Ratings
0,",how do I activate ms office lifetime0l",",how do I activate ms office lifetime",3.0 out of 5 stars
1,Bluetooth not working,Bluetooth not working,1.0 out of 5 stars
2,Hangs soo much that you will feel frustrated,"Hangs soo much that you feel frustrated,<br>La...",1.0 out of 5 stars
3,Running smoothly,No heating issue was found....!!<br>working ve...,5.0 out of 5 stars
4,A beast in the budget price,I purchased this laptop using exchange offer a...,5.0 out of 5 stars
...,...,...,...
108298,Worthless,One of the worst products the screen protector...,1
108299,Wonderful,Product quality is normal.,4
108300,Wonderful,Product quality is normal.,4
108301,Nice product,Awsm\nSticker is good\nCover is good\nKeyboard...,4


In [14]:
# Saving the data into csv file
reviews.to_csv("Review_Rating.csv",index=False)

In [15]:
# Saving the data into excel file
reviews.to_excel("Review_Ratings.xlsx",index=False)